In [11]:
#init
import pandas as pd 
import numpy as np 

In [12]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 150)

In [13]:
raw_df = pd.read_csv("../data/ufc_tiger_millionaire_raw.csv")

In [14]:
#resolve the mixed date formats.
raw_df['date']=pd.to_datetime(raw_df['date'], format='mixed')
#sort by latest fights because sure why not
raw_df=raw_df.sort_values('date', ascending=False)

In [15]:
#doesn't seem like there's any duplicates

raw_df[['R_fighter', 'B_fighter', 'date']].drop_duplicates().shape[0] == raw_df.shape[0]

True

In [16]:
#drop location column and replace it with city/region/country columns.
#assuming everything is "city, region, country" strings
locations_df=raw_df['location'].str.split(",", expand=True).rename(columns={0: "city", 1: "region", 2: "country"})
#strip the whitespace in each of the columns
for col in locations_df.columns:
    locations_df[col] = locations_df[col].str.strip()

#sike some locations were recorded only as city/country, which means country got assigned to region column. fill na country slots with this.

locations_df['country']=locations_df['country'].fillna(locations_df['region'])

df = pd.concat([raw_df.drop(columns=['location', 'country']), locations_df], axis=1)

In [17]:
#i spot tested that city/region/country doesn't have any misspelled or inconsistently capitalized bullshit in it.

In [18]:

#there is exactly one busted odds value. we could probably just look this up., but i'll drop it for now.
df[df['R_odds'].isnull()]

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,Winner,title_bout,weight_class,gender,no_of_rounds,B_current_lose_streak,B_current_win_streak,B_draw,B_avg_SIG_STR_landed,B_avg_SIG_STR_pct,B_avg_SUB_ATT,B_avg_TD_landed,B_avg_TD_pct,B_longest_win_streak,B_losses,B_total_rounds_fought,B_total_title_bouts,B_win_by_Decision_Majority,B_win_by_Decision_Split,B_win_by_Decision_Unanimous,B_win_by_KO/TKO,B_win_by_Submission,B_win_by_TKO_Doctor_Stoppage,B_wins,B_Stance,B_Height_cms,B_Reach_cms,B_Weight_lbs,R_current_lose_streak,R_current_win_streak,R_draw,R_avg_SIG_STR_landed,R_avg_SIG_STR_pct,R_avg_SUB_ATT,R_avg_TD_landed,R_avg_TD_pct,R_longest_win_streak,R_losses,R_total_rounds_fought,R_total_title_bouts,R_win_by_Decision_Majority,R_win_by_Decision_Split,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_wins,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,R_age,B_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,constant_1,B_match_weightclass_rank,R_match_weightclass_rank,R_Women's Flyweight_rank,R_Women's Featherweight_rank,R_Women's Strawweight_rank,R_Women's Bantamweight_rank,R_Heavyweight_rank,R_Light Heavyweight_rank,R_Middleweight_rank,R_Welterweight_rank,R_Lightweight_rank,R_Featherweight_rank,R_Bantamweight_rank,R_Flyweight_rank,R_Pound-for-Pound_rank,B_Women's Flyweight_rank,B_Women's Featherweight_rank,B_Women's Strawweight_rank,B_Women's Bantamweight_rank,B_Heavyweight_rank,B_Light Heavyweight_rank,B_Middleweight_rank,B_Welterweight_rank,B_Lightweight_rank,B_Featherweight_rank,B_Bantamweight_rank,B_Flyweight_rank,B_Pound-for-Pound_rank,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,city,region,country
32,Rong Zhu,Brandon Jenkins,NaN,195,NaN,195.0,2021-09-18,Red,False,Lightweight,MALE,3,0,0,0,2.81,0.39,0.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,Switch,182.88,180.34,155,1,0,0,4.33,0.53,0.0,4.56,0.75,0,1,3,0,0,0,0,0,0,0,0,Orthodox,175.26,180.34,155,21,29,-1,0,0,0,-1,-3,0,0,0,7.62,0.0,8,-1.52,0.0,-4.56,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neither,KO/TKO,Punches,3.0,4:35,875.0,NaN,500.0,NaN,1600.0,NaN,450.0,Las Vegas,Nevada,USA


In [19]:
df=df[df['R_odds'].notnull()]

In [20]:
#i don't trust that the stat data was mined as of the date of of a given fight properly, but i'm too lazy to get into that right now
#will trust odds to EV calc is correct for now, but should verify.

In [21]:
#rename columns to be less garby
#will keep syntax of r_ and b_ for red and blue corner respectively.
#lower case everything
#hardcoding some dumb symbol replacement
def rename_map(col_name):
    col_name=col_name.lower()
    #replace apostrephes with empty string
    col_name=col_name.replace("\'","")
    col_name=col_name.replace("/","_")
    #replace hyphens with underscore
    col_name=col_name.replace("-","_")
    col_name=col_name.replace(" ", "_")
    return col_name
    

In [22]:
df=df.rename(columns=rename_map)

In [23]:
#b_stance had some whitespace bullshit
df['b_stance']=df['b_stance'].str.strip()
#i don't know why this column exists
df=df.drop(columns=['constant_1'])

In [24]:
# #fill na ranks with "999"

# #not sure if this is actually a good idea or not, but would rather not have nulls.
# rank_cols=[col for col in df.columns if (col.endswith("rank") and (col != 'better_rank')) ]
# df[rank_cols]=df[rank_cols].fillna(999)
# #also, i suspect ranks are underpopulated but don't wanna double check that right now.

In [25]:
#casting some columns that were imported as float because of nulls to Int64
rank_cols=[col for col in df.columns if (col.endswith("rank") and (col != 'better_rank')) ]
Int64_cast_cols=['b_odds', 'r_odds', 'finish_round', 'total_fight_time_secs', 'r_dec_odds', 'b_dec_odds', 'r_sub_odds', 'b_sub_odds', 'r_ko_odds', 'b_ko_odds'] + rank_cols
df[Int64_cast_cols]=df[Int64_cast_cols].astype('Int64')

In [26]:
df.head()

,r_fighter,b_fighter,r_odds,b_odds,r_ev,b_ev,date,winner,title_bout,weight_class,gender,no_of_rounds,b_current_lose_streak,b_current_win_streak,b_draw,b_avg_sig_str_landed,b_avg_sig_str_pct,b_avg_sub_att,b_avg_td_landed,b_avg_td_pct,b_longest_win_streak,b_losses,b_total_rounds_fought,b_total_title_bouts,b_win_by_decision_majority,b_win_by_decision_split,b_win_by_decision_unanimous,b_win_by_ko_tko,b_win_by_submission,b_win_by_tko_doctor_stoppage,b_wins,b_stance,b_height_cms,b_reach_cms,b_weight_lbs,r_current_lose_streak,r_current_win_streak,r_draw,r_avg_sig_str_landed,r_avg_sig_str_pct,r_avg_sub_att,r_avg_td_landed,r_avg_td_pct,r_longest_win_streak,r_losses,r_total_rounds_fought,r_total_title_bouts,r_win_by_decision_majority,r_win_by_decision_split,r_win_by_decision_unanimous,r_win_by_ko_tko,r_win_by_submission,r_win_by_tko_doctor_stoppage,r_wins,r_stance,r_height_cms,r_reach_cms,r_weight_lbs,r_age,b_age,lose_streak_dif,win_streak_dif,longest_win_streak_dif,win_dif,loss_dif,total_round_dif,total_title_bout_dif,ko_dif,sub_dif,height_dif,reach_dif,age_dif,sig_str_dif,avg_sub_att_dif,avg_td_dif,empty_arena,b_match_weightclass_rank,r_match_weightclass_rank,r_womens_flyweight_rank,r_womens_featherweight_rank,r_womens_strawweight_rank,r_womens_bantamweight_rank,r_heavyweight_rank,r_light_heavyweight_rank,r_middleweight_rank,r_welterweight_rank,r_lightweight_rank,r_featherweight_rank,r_bantamweight_rank,r_flyweight_rank,r_pound_for_pound_rank,b_womens_flyweight_rank,b_womens_featherweight_rank,b_womens_strawweight_rank,b_womens_bantamweight_rank,b_heavyweight_rank,b_light_heavyweight_rank,b_middleweight_rank,b_welterweight_rank,b_lightweight_rank,b_featherweight_rank,b_bantamweight_rank,b_flyweight_rank,b_pound_for_pound_rank,better_rank,finish,finish_details,finish_round,finish_round_time,total_fight_time_secs,r_dec_odds,b_dec_odds,r_sub_odds,b_sub_odds,r_ko_odds,b_ko_odds,city,region,country
0,Thiago Santos,Johnny Walker,-150,130,66.666667,130.000000,2021-10-02,Red,False,Light Heavyweight,MALE,5,0,1,0,3.42,0.59,0.7,0.24,1.00,4,2,11,0,0,0,1,4,0,0,5,Orthodox,198.12,208.28,205,3,0,0,3.95,0.48,0.1,0.61,0.36,4,8,43,1,0,0,2,11,0,0,13,Orthodox,187.96,193.04,205,37,29,-3,1,0,-8,-6,-32,-1,-7,0,10.16,15.24,-8,-0.53,0.6,-0.37,1,10,5,<NA>,<NA>,<NA>,<NA>,<NA>,5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,10,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Red,U-DEC,NaN,5,5:00,1500,800,900,2000,1600,-110,175,Las Vegas,Nevada,USA
6,Bethe Correia,Karol Rosa,500,-720,500.000000,13.888889,2021-10-02,Blue,False,Women's Bantamweight,FEMALE,3,0,3,0,7.88,0.54,0.0,1.50,0.50,3,0,9,0,0,1,2,0,0,0,3,Orthodox,165.10,170.18,135,1,0,1,4.37,0.43,0.0,0.39,0.18,3,5,30,1,0,2,2,1,0,0,5,Orthodox,165.10,162.56,135,38,26,-1,3,0,-2,-5,-21,-1,-1,0,0.00,7.62,-12,3.51,0.0,1.11,1,15,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,15,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,Blue,U-DEC,NaN,3,5:00,900,700,-200,2500,600,1600,400,Las Vegas,Nevada,USA
1,Alex Oliveira,Niko Price,170,-200,170.000000,50.000000,2021-10-02,Blue,False,Welterweight,MALE,3,2,0,0,5.16,0.42,0.8,0.79,0.22,2,5,24,0,0,0,0,4,2,0,6,Orthodox,182.88,193.04,170,2,0,0,2.97,0.51,0.5,2.27,0.37,4,8,44,0,0,1,3,4,3,0,11,Orthodox,180.34,193.04,170,33,32,0,0,-2,-5,-3,-20,0,0,-1,2.54,0.00,-1,2.19,0.3,-1.48,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,neither,U-DEC,NaN,3,5:00,900,450,350,700,1100,550,120,Las Vegas,Nevada,USA
10,Alejandro Perez,Johnny Eduardo,-300,235,33.333333,235.000000,2021-10-02,Red,False,Bantamweight,MALE,3,2,0,0,2.95,0.39,0.4,0.22,0.20,2,4,14,0,0,0,1,2,0,0,3,Orthodox,170.18,180.34,135,2,0,1,3.45,0.35,0.4,0.61,0.23,4,3,25,0,0,1,3,3,0,0,7,Orthodox,167.64,165.10,145,32,43,0,0,-2,-4,1,-11,0,-1,0,2.54,15.24,11,-0.50,0.0,-0.39,1,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,neith

In [27]:
#could clean these. not sure if "punch" vs "punches" is actually different, etc.
df['finish_details'].value_counts()

finish_details
Punch                 507
Punches               466
Rear Naked Choke      331
Guillotine Choke      146
Kick                  119
Armbar                 95
Knee                   67
Elbows                 60
Arm Triangle           58
Triangle Choke         46
Elbow                  30
D'Arce Choke           26
Kimura                 21
Flying Knee            20
Knees                  16
Other - Choke          14
Anaconda Choke         14
Kneebar                12
Heel Hook              11
Spinning Back Fist      6
Slam                    6
Spinning Back Kick      6
Other - Lock            5
Neck Crank              5
North-South Choke       4
Injury                  3
Omoplata                2
Kicks                   2
Ankle Lock              1
Peruvian Necktie        1
Keylock                 1
Name: count, dtype: int64

In [28]:
#aidan, i don't actually want a csv of this. this is for you.
#df.to_csv("../data/ufc_tiger_millionaire_proc.csv", index=False)